In [23]:
import pandas as pd
import csv
import os
import numpy as np
import gurobipy as gp
from gurobipy import LinExpr, GRB
from itertools import permutations
from itertools import combinations
import googlemaps as gmaps
from datetime import datetime


gmaps = gmaps.Client(key= # Google Maps API key here)

-------------------------------------------------------------------------------------------------------------------

### Parameters

In [24]:
## Fomulate ride, wait and distance constants

import re

########################## Create Sets ###############################

# All Rides
capitals = ['Entrance', 
            'Its a Small World 1', 'Its a Small World 2', 
            'Space Mountain 1', 'Space Mountain 2', 
            'Star Wars: Rise of Resistance 1', 'Star Wars: Rise of Resistance 2', 
            'Indiana Jones Adventure 1', 'Indiana Jones Adventure 2', 
            'Pirates of the Caribbean 1', 'Pirates of the Caribbean 2', 
            'Splash Mountain 1', 'Splash Mountain 2', 
            'Big Thunder Mountain Railway 1', 'Big Thunder Mountain Railway 2', 
            'Matterhorn Bobsleds 1', 'Matterhorn Bobsleds 2', 
            'Fireworks',
            'Exit']

# Main Rides and Extra Attractions
capitals_minus = ['Its a Small World 1', 'Its a Small World 2', 
            'Space Mountain 1', 'Space Mountain 2', 
            'Star Wars: Rise of Resistance 1', 'Star Wars: Rise of Resistance 2', 
            'Indiana Jones Adventure 1', 'Indiana Jones Adventure 2', 
            'Pirates of the Caribbean 1', 'Pirates of the Caribbean 2', 
            'Splash Mountain 1', 'Splash Mountain 2', 
            'Big Thunder Mountain Railway 1', 'Big Thunder Mountain Railway 2', 
            'Matterhorn Bobsleds 1', 'Matterhorn Bobsleds 2',
            'Fireworks']
 
# Main Rides
first_ride = ['Its a Small World 1', 'Space Mountain 1', 
                  'Star Wars: Rise of Resistance 1', 
                  'Indiana Jones Adventure 1', 'Pirates of the Caribbean 1', 
                  'Splash Mountain 1', 'Big Thunder Mountain Railway 1', 
                  'Matterhorn Bobsleds 1']

# Duplicate of Main Rides
second_ride = ['Its a Small World 2', 'Space Mountain 2', 
                  'Star Wars: Rise of Resistance 2', 
                  'Indiana Jones Adventure 2', 'Pirates of the Caribbean 2', 
                  'Splash Mountain 2', 'Big Thunder Mountain Railway 2', 
                  'Matterhorn Bobsleds 2']

# Activities
capitals_food = ['Entrance', 
                 'Its a Small World 1', 'Its a Small World 2', 
            'Space Mountain 1', 'Space Mountain 2', 
            'Star Wars: Rise of Resistance 1', 'Star Wars: Rise of Resistance 2', 
            'Indiana Jones Adventure 1', 'Indiana Jones Adventure 2', 
            'Pirates of the Caribbean 1', 'Pirates of the Caribbean 2',
            'Splash Mountain 1', 'Splash Mountain 2', 
            'Big Thunder Mountain Railway 1', 'Big Thunder Mountain Railway 2',
            'Matterhorn Bobsleds 1', 'Matterhorn Bobsleds 2', 
                'Fireworks',
                 'Exit',
                 'Plaza Inn - L', 
                 'Galactic Grill - L', 
                 "Daisy's Dinner - L", 
                 'Blue Bayou Restaurant - L', 
                 'Bengal Barbeque - L',
                 'The Golden Horseshoe - L', 
                 'Red Rose Tavern - L', 
                 'Troubadour Tavern - L',
                 'Plaza Inn - D', 
                 'Galactic Grill - D', 
                 "Daisy's Dinner - D", 
                 'Blue Bayou Restaurant - D', 
                 'Bengal Barbeque - D',
                 'The Golden Horseshoe - D', 
                 'Red Rose Tavern - D', 
                 'Troubadour Tavern - D']

# Food Option Names 
food = ['Plaza Inn - L', 
        'Galactic Grill - L', 
        "Daisy's Dinner - L", 
        'Blue Bayou Restaurant - L', 
        'Bengal Barbeque - L',
        'The Golden Horseshoe - L', 
        'Red Rose Tavern - L', 
        'Troubadour Tavern - L',
        'Plaza Inn - D', 
        'Galactic Grill - D', 
        "Daisy's Dinner - D", 
        'Blue Bayou Restaurant - D', 
        'Bengal Barbeque - D',
        'The Golden Horseshoe - D', 
        'Red Rose Tavern - D', 
        'Troubadour Tavern - D']

########################## Parameters ###############################

# Average wait time
av_wt = [0,
 14.176750419333157, 14.176750419333157, 
 49.2742706178715, 49.2742706178715, 
 100.52732538989471, 100.52732538989471,
 42.74889237079236, 42.74889237079236, 
 23.366666395373894, 23.366666395373894,
 30.709718187729766, 30.709718187729766,
 33.67079740724337, 33.67079740724337, 
 37.825957101015725, 37.825957101015725,
        10,
 0, 
 0, 
 0, 
 0, 
 0, 
 0, 
 0, 
 0, 
 0, 
 0, 
 0, 
 0,
 0,
 0, 
 0, 
 0,
 0]

# Ride Time
ride = [0, 14, 14, 
        5, 5, 
        18, 18,
        10, 10,
        10, 10,
        11, 11,
        3.5, 3.5,
        4, 4, 
    20,
        0, 
        45, 45, 45, 45, 45, 45, 45, 45, 65, 65, 65, 65, 65, 65, 65, 65]

# Create Dictionaries for wait and ride time
wait = {}
ride_time = {}

for i in range(0,35):
    wait[capitals_food[i]] = av_wt[i]
    ride_time[capitals_food[i]] = ride[i]

# Prioritization
# larger reward for better rides
reward_val = [5, 5, 7, 7, 8, 8, 6, 6, 4, 4, 1, 1, 3, 3, 2, 2, 9, 9]

reward = {}
for i in range(0, 17):
    reward[capitals_minus[i]]= reward_val[i]
    
    
    
# DISTANCE MATRIX

distance_matrix = pd.read_csv('distance_matrix_trides.csv',index_col=0)

# convert to dictionary
dist = {(row['Origin'], row['Destination']): row['Walk Time'] for _, row in distance_matrix.iterrows()}

# make matching origin and destinations larger
for key,value in dist.items():
    if key[0] == key[1]:
        dist[key] = 100

# Special Case for the double entrance value
dist['Entrance', 'Exit'] = 100
dist['Exit', 'Entrance'] = 100

-------------------------------------------------------------------------------------------------------------------

## Problem Formulation

In [25]:
import gurobipy as gp
from gurobipy import LinExpr, GRB

m = gp.Model()

# [Parameter] large number double that of the ub on time
M = 2000

# [Parameter] 16 hours of park opening
T = 960


# [Variable] Arch connections: is ride 'i' adjacent to ride 'j' on the route?
vars = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='x')

# [Variable] Timestamp: What time does the guest arrive at each ride?
t = m.addVars(capitals_food, lb = 0, ub = T)

# [Variable] Check for visiting a lunch food stop
food_stop = m.addVars(food, vtype=GRB.BINARY)

# [Variable] Check for if visiting a ride
ride_stop = m.addVars(capitals_minus, vtype=GRB.BINARY)



# [Constraint] Input/Output constraints to create a route with no subtours where all rides visited once
for i in capitals_food:
    input = 0
    output = 0
    # sum across all combinations of pairings
    for j in capitals_food:
        if i != j:
            input += vars[j, i]
            output += vars[i,j]
    if i == 'Entrance':
        # Entrance should have not input and 1 output
        m.addConstr(output == 1)
    elif i == 'Exit':
        # Exit should have 1 input and no outputs
        m.addConstr(input ==1)
    else: 
        m.addConstr(input == output)
        # must ride each ride once
        if i in first_ride:
            m.addConstr(input == 1)
            m.addConstr(input == ride_stop[i])
        elif i not in food:
            m.addConstr(input <= 1) 
            m.addConstr(input == ride_stop[i])
        else: 
            m.addConstr(input == food_stop[i])

# [Constraint] Must visit 2 food locations            
m.addConstr(food_stop.sum() == 2)        
 
# [Constraint] No Repeats
j = 0
for i in first_ride:
        m.addConstr(vars[i, second_ride[j]] == 0)
        m.addConstr(vars[second_ride[j],i] == 0)
        j = j + 1
        
    
# [Constraint] total timing: must be less than the total hours of the park
m.addConstr(t['Exit']-t['Entrance']<= T)

# [Constraint] Start time at 0
m.addConstr(t['Entrance'] == 0)

# [Constraint] Time must forward for rides and food
for i in capitals_food:
    for j in capitals_food:
        if i != j:
            m.addConstr(t[j] >= t[i] - M*(1-vars[i, j]) + dist[i, j] + wait[i] + ride_time[i])
            
            
        
# [Constraint] Must Visit lunch between 11 and 2  
total1 = 0
for i in range(0, 8):
    f = food[i]
    total1 = total1 + food_stop[f]
    m.addConstr(360 >= t[f])
    m.addConstr(180 <= t[f])

# [Constraint] Must have one lunch stop and one dinner stop
m.addConstr(total1 == 1)
    
# [Constraint] Must Visit dinner between 5 and 8  
for i in range(8, 16):
    f = food[i]
    m.addConstr(720 >= t[f])
    m.addConstr(540 <= t[f])
    

# [Constraint] Cannot visit the same food place twice (unique lunch and dinner)
for i in range(0,8):
    f = food[i]
    g = food[i+8]
    m.addConstr(food_stop[f]+food_stop[g]<=1)
    
# # [Constraint] Visit parade between 2pm and 4 pm
# m.addConstr(t['Parade']>= 360)
# m.addConstr(t['Parade']<= 480)

# [Constraint] Visit Fireworks between 8:40and 9
m.addConstr(t['Fireworks'] >= 780) # 9:00 pm
m.addConstr(t['Fireworks'] <= 840) # 10:00 pm
    
    
    
# [Objective] Set objective to minimize the time guest arrives at Exit and to minimize the walking distance
m.ModelSense = GRB.MINIMIZE
m.setObjective(-gp.quicksum(ride_stop[i]*reward[i] for i in capitals_minus), 0)
m.setObjectiveN(gp.quicksum(vars[i, j]*dist[i,j] for i, j in vars.keys()), 1)

In [26]:
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1337 rows, 1293 columns and 7708 nonzeros
Model fingerprint: 0xfac29dea
Variable types: 35 continuous, 1258 integer (1258 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 2e+03]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives (1 combined) ...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a model with 1337 rows, 1293 columns and 7708 nonzeros
Model fingerprint: 0x155b9bfb
Variable types: 3

In [31]:
vals = m.getAttr('x', vars)
t = m.getAttr('x', t)

selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

-------------------------------------------------------------------------------------------------------------------

## Reward Total

In [32]:
# Find Total Reward of Route
total = 0
for i in capitals_minus:
    total = total + (ride_stop[i].X* reward[i])
    
print('Total Reward: ', total)

Total Reward:  78.0


-------------------------------------------------------------------------------------------------------------------

## Walking Distance

In [33]:
d = distance_matrix['Distance Walk'] # add to data fram

# Change enteries in meters to km
for i in range(0,1225):
    if d[i] >= 1.0:
        temp = d[i]
        d[i] = temp/1000
        
distance_matrix['Distance Walk'] = d # add to data fram

# convert to dictionary
walk = {(row['Origin'], row['Destination']): row['Distance Walk'] for _, row in distance_matrix.iterrows()}

# Find total walking distance
total = 0
for i, j in dist.keys():
    total = total + walk[i,j]*vals[i,j]

# Convert to Miles
miles = total/1.609

print('Total miles walked: ', miles)

Total miles walked:  2.3772529521441887


/var/folders/40/khdb0r0d62q_c42_9lq8c1cc0000gn/T/ipykernel_893/830130446.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[i] = temp/1000


-------------------------------------------------------------------------------------------------------------------

## Find Full Path

In [34]:
# find path not including Entrance and Exit
visited = {}

for i in capitals_food:
    visited[i] = 0

# Start at Entrance
path = []
path.append('Entrance')
visited['Entrance'] = 1


# Where to go from Entrance
for i in capitals_food:
        if vals['Entrance', i] == 1:
            visited[i] = 1
            path.append(i)
            break

# Continue from Entrance on path
while len(path) < 19:
    index = len(path) - 1
    temp = path[index]
    for i in capitals_food:
        if visited[i] == 0:
             if vals[temp, i] == 1:
                visited[i] = 1
                path.append(i)
path

['Entrance',
 'Pirates of the Caribbean 1',
 'Indiana Jones Adventure 2',
 'Pirates of the Caribbean 2',
 'Indiana Jones Adventure 1',
 'Big Thunder Mountain Railway 1',
 'The Golden Horseshoe - L',
 'Big Thunder Mountain Railway 2',
 'Star Wars: Rise of Resistance 2',
 'Splash Mountain 1',
 'Star Wars: Rise of Resistance 1',
 'Its a Small World 2',
 'Troubadour Tavern - D',
 'Its a Small World 1',
 'Matterhorn Bobsleds 1',
 'Space Mountain 1',
 'Fireworks',
 'Space Mountain 2',
 'Exit']

-------------------------------------------------------------------------------------------------------------------

## Itenerary

In [36]:
# [Timestamp] Print out the timestamp that a guest arrives at each ride on the route
import datetime

timestamp = {}

for i in t:
    time = t[i] + 8*60 #gets zero to be 8am
    # Calculate PM times
    if time > 720:
        half = "pm"
        hour, minutes = divmod(time, 60)
        correct_hour = hour - 12
        if correct_hour == 0:
            correct_hour = 12
    # Calculate AM times
    else: 
        half = "am"
        correct_hour, minutes = divmod(time, 60)
        # format min less than 10 so there is a zero before the number
    if minutes < 10:
        minutes = '0{min}'.format(min = int(minutes))
    else:
        minutes = int(minutes)
    result = '{hour}:{mins} {xm}'.format(hour = int(correct_hour), mins = minutes, xm = half)
    timestamp[i] = result

    
# [Iternerary] print out the time stamps in order of the path
iten = {}

for i in path:
    iten[i] = timestamp[i]
    
iten

{'Entrance': '8:00 am',
 'Pirates of the Caribbean 1': '8:05 am',
 'Indiana Jones Adventure 2': '8:39 am',
 'Pirates of the Caribbean 2': '9:33 am',
 'Indiana Jones Adventure 1': '10:07 am',
 'Big Thunder Mountain Railway 1': '11:02 am',
 'The Golden Horseshoe - L': '11:40 am',
 'Big Thunder Mountain Railway 2': '12:26 pm',
 'Star Wars: Rise of Resistance 2': '1:08 pm',
 'Splash Mountain 1': '3:10 pm',
 'Star Wars: Rise of Resistance 1': '4:06 pm',
 'Its a Small World 2': '6:12 pm',
 'Troubadour Tavern - D': '6:41 pm',
 'Its a Small World 1': '7:47 pm',
 'Matterhorn Bobsleds 1': '8:17 pm',
 'Space Mountain 1': '9:02 pm',
 'Fireworks': '10:00 pm',
 'Space Mountain 2': '11:00 pm',
 'Exit': '12:00 pm'}

-------------------------------------------------------------------------------------------------------------------

## Find Path without Entrance or Exit

In [37]:
# find path not including Entrance and Exit
visited = {}

for i in capitals_food:
    visited[i] = 0

# Start at Entrance
path = []
visited['Entrance'] = 1


# Where to go from Entrance
for i in capitals_food:
        if vals['Entrance', i] == 1:
            visited[i] = 1
            path.append(i)
            break

# Continue from Entrance on path
while len(path) < 17:
    index = len(path) - 1
    temp = path[index]
    for i in capitals_food:
        if visited[i] == 0:
             if vals[temp, i] == 1:
                visited[i] = 1
                path.append(i)
path

['Pirates of the Caribbean 1',
 'Indiana Jones Adventure 2',
 'Pirates of the Caribbean 2',
 'Indiana Jones Adventure 1',
 'Big Thunder Mountain Railway 1',
 'The Golden Horseshoe - L',
 'Big Thunder Mountain Railway 2',
 'Star Wars: Rise of Resistance 2',
 'Splash Mountain 1',
 'Star Wars: Rise of Resistance 1',
 'Its a Small World 2',
 'Troubadour Tavern - D',
 'Its a Small World 1',
 'Matterhorn Bobsleds 1',
 'Space Mountain 1',
 'Fireworks',
 'Space Mountain 2']

-------------------------------------------------------------------------------------------------------------------

# Mapping

### Load Coordinates

In [ ]:
# create dictionary of latitude and longitude for each ride
lat_long = pd.read_csv('Lat_Long_trides.csv',index_col=0)

# ride name as key with (lat, long) as values
coordinates = {(row['Ride']): (row['Latitude'], row['Longitude']) for _, row in lat_long.iterrows()}

# because coordinates are similar, change to slightly different coordinates for marker on map
coordinates['Pirates of the Caribbean 1'] = (33.811757, -117.9207289)
coordinates['Pirates of the Caribbean 2'] = (33.811757, -117.9207289)
coordinates['Exit'] = ((33.809997, -118.9189785))

### Map Route and Stops

In [ ]:
# Map the route using google maps static maps

# [Waypoints] use path to create waypoints that the route must go through
waypoints = []

for i in path:
    waypoints.append(coordinates[i])
    
    
# [Direction] get step by step directions from Entrance to Exit through waypoints
results = gmaps.directions(origin = coordinates['Entrance'], destination = coordinates['Entrance'], mode = 'walking', waypoints= waypoints)


# [Mapping] Plot on map
marker_points = []
waypoints = []

#extract the location (lat and long) of each turn from the previous directions function
for leg in results[0]["legs"]:
    # adds a marker for each starting waypoint
    leg_start_loc = leg["start_location"]
    marker_points.append(f'{leg_start_loc["lat"]},{leg_start_loc["lng"]}')
    for step in leg["steps"]:
        # adds a point for each turn on the way to the next ride on path
        end_loc = step["end_location"]
        waypoints.append(f'{end_loc["lat"]},{end_loc["lng"]}')
# adds a marker for the last ride on path
last_stop = results[0]["legs"][-1]["end_location"]
marker_points.append(f'{last_stop["lat"]},{last_stop["lng"]}')
 
# labels all marker points (all rides on path) with a letter
markers = [ "color:red|size:mid|label:" + chr(65+i) + "|" 
           + r for i, r in enumerate(marker_points)]

# Change color of markers and move the second visit to each ride over so it can be seen
markers[1] = 'color:purple|size:mid|label:C|33.811357, -117.9207289' # Pirates 1
markers[2] = 'color:white|size:mid|label:B|33.8117056,-117.9206731' # Indiana Jones 1
markers[3] = 'color:purple|size:mid|label:E|33.811357, -117.9204289'# Pirates 2
markers[4] = 'color:white|size:mid|label:D|33.8117056,-117.9203731' # Indiana Jones 2
markers[5] = 'color:yellow|size:mid|label:F|33.8124758,-117.9203355' # Big Thunder 1
markers[6] = 'color:green|size:mid|label:G|33.8121265,-117.9202446' # LUNCH
markers[7] = 'color:yellow|size:mid|label:H|33.8124758,-117.9200355' #Big Thunder 2
markers[8] = 'color:black|size:mid|label:I|33.8137067,-117.9231294' # Star Wars 1
markers[9] = 'color:red|size:mid|label:J|33.8122899,-117.9224001' # Splash
markers[10] = 'color:black|size:mid|label:K|33.8137067,-117.9228294' # Star Wars 2
markers[11] = 'color:orange|size:mid|label:L|33.8143046,-117.9178047' # Small World 1
markers[12] = 'color:green|size:mid|label:M|33.81455529999999,-117.9187207' # Dinner
markers[13] = 'color:orange|size:mid|label:N|33.8143046,-117.9175047' # Small World 2
markers[14] = 'color:red|size:mid|label:O|33.8130703, -117.9178203' # Matterhorn
markers[15] = 'color:blue|size:mid|label:P|33.8116723,-117.9173565' # Space 1
markers[16] = 'color:red|size:mid|label:Q|33.8127953,-117.9189839' # Fireworks
markers[17] = 'color:blue|size:mid|label:R|33.8116723,-117.9170565' # Space 2


# [Print Map]
result_map = gmaps.static_map(
                 center = [33.812252236772736, -117.91894201571984],
                 scale=2, 
                 zoom=16.95,
                 size=[640, 640], 
                 format="jpg", 
                 maptype="satellite",
                 markers=markers,
                 path="geodesic:TRUE|color:0x0000ff|weight:2|" + "|".join(waypoints))

# [Save Map]
with open('[Thesis]Prioritization_Complication_Map.jpg', 'wb') as img:
    for chunk in result_map:
        img.write(chunk)